# ViT (Vision Transformer)

在Google团队在2017提出《Attention is all you need》后，self-attention机制在NLP领域成为SOTA方法。自然有一种想法，是否可以将Transformer处理机器翻译任务的方法流程迁移到图像处理来，于是在2020年Google又一次提出了Vision Transformer这一力作。

**ViT原论文中最核心的结论是，当拥有足够多的数据进行预训练的时候，ViT的表现就会超过CNN，突破transformer缺少归纳偏置的限制，可以在下游任务中获得较好的迁移效果。**

但是当训练数据集不够大的时候，ViT的表现通常比同等大小的ResNets要差一些，因为Transformer和CNN相比缺少归纳偏置（inductive bias），即一种先验知识，提前做好的假设。CNN具有两种归纳偏置，一种是局部性（locality/two-dimensional neighborhood structure），即图片上相邻的区域具有相似的特征；一种是平移不变形（translation equivariance）。当CNN具有以上两种归纳偏置，就有了很多先验信息，需要相对少的数据就可以学习一个比较好的模型。

## ViT 的结构

论文中提出的ViT的结构如下图所示。相较于Transformer其去掉了Decoder部分，取而代之的是MLPHead，其是一个只含有两层全连接层的浅层感知机，激活函数使用的是GELU。（在Transformer的ffw层使用的是ReLU。）而输入部分还是采用的是序列输入，因此需要把图片分为数个Patch，再将每个Patch投影为固定长度的Embedding向量再输入Encoder中，参考[知乎文章详解ViT](https://zhuanlan.zhihu.com/p/445122996)，下面具体讲解ViT的任务流程。

<div align = "center">
    <img src = ./images/ViT.png width = 80%>
</div>

1. **Patch Embedding** 首先是输入图片被切成一个一个的Patch，这一步论文中采用的是卷积的方法。例如输入图像的大小是 224x224x3 卷积操作将输入图像切分为 16x16 大小的Patch，那么就会产生196个Patch（即产生维度为 196x16x16x3 的特征序列），然后经过线性投影会产生维度大小为 196x768 的Token。由于ViT处理的事图像分类的问题，还需要在Token的第一行加上一个可学习的Class Embedding，因此此时的Token维度变为 197x768 ，到目前为止，已经通过patch embedding将一个视觉问题转化为了一个seq2seq问题。
2. **Position Embedding** 和Transformer一样，图像不同Patch的位置信息对最后预测结果的正确与否具有重要影响。因此为每一个经过线性投影的Patch加上位置表征的Position Embedding自然很重要，ViT不同于Transformer采用的人工手写正余弦位置表征的方法，采用可学习的1D position embedding。其位置编码可以理解为一张表，表一共有N行，N的大小和输入序列相同（197），每一列代表一个向量，向量的维度和输入序列embedding的维度相同（768）。**注意位置编码的操作是sum，而不是concat。**加入位置编码信息之后，维度依然是197x768。
3. **Transformer Encoder** 这里的Encoder结构和17年提出的Transformer架构基本保持一致，如果一定要找不同的话只有最后的线性全连接层激活函数变成了GELU，图中的名字也从Feed forward变成了MLP。进入Encoder首先经过一个Layer norm层，其调整一个layer层的均值为0，方差为1，且不改变数据的维度，因此LN输出的数据维度还是 197x768.然后是进入多头自注意力，先将输入映射到q，k，v，如果只有一个头，qkv的维度都是197x768，如果有12个头（768/12=64），则qkv的维度是197x64，一共有12组qkv，最后再将12组qkv的输出拼接起来，输出维度是197x768，然后在过一层LN，维度依然是197x768。
4. **MLP** 两层全连接层将维度放大再缩小回去，197x768放大为197x3072，再缩小变为197x768。

一个Encoder block之后维度依然和输入相同，都是197x768，因此可以堆叠多个block。最后会将特殊字符cls对应的输出$Z_{L}^{0}$作为encoder的最终输出 ，代表最终的image presentation（另一种做法是不加cls字符，对所有的tokens的输出做一个平均），如下图公式(4)，后面接一个MLP进行图片分类.

<div align = "center">
    <img src = ./images/Vit_formula.png width = 80%>
</div>

另外，关于是否要在Token中加入class embedding，论文的附录中也有说明：通过实验表明，同样可以使用average pooling，原文ViT是为了尽可能是模型结构接近原始的Transformer，所以采用了类似于BERT的做法，加入特殊字符。并且从图中可以看出，使用平均池化的方法进行图片分类时，不同学习率所对应的ACC相距较大，因此要注意调参。

<div align = "center">
    <img src = ./images/yon_cla.png width = 80%>
</div>

关于positional encoding

1-D 位置编码：例如3x3共9个patch，patch编码为1到9

2-D 位置编码：patch编码为11,12,13,21,22,23,31,32,33，即同时考虑X和Y轴的信息，每个轴的编码维度是D/2

实际实验结果表明，不管使用哪种位置编码方式，模型的精度都很接近，甚至不使用位置编码，模型的性能损失也没有特别大。原因可能是ViT是作用在image patch上的，而不是image pixel，对网络来说这些patch之间的相对位置信息很容易理解，所以使用什么方式的位置编码影响都不大。

<div align = "center">
    <img src = ./images/vit_position.png width = 80%>
</div>

关于输入图片大小

通常在一个很大的数据集上预训练ViT，然后在下游任务相对小的数据集上微调，已有研究表明在分辨率更高的图片上微调比在在分辨率更低的图片上预训练效果更好（It is often beneficial to fine-tune at higher resolution than pre-training）（参考2019-NIPS-Fixing the train test resolution discrepancy）

当输入图片分辨率发生变化，输入序列的长度也发生变化，虽然ViT可以处理任意长度的序列，但是预训练好的位置编码无法再使用（例如原来是3x3，一种9个patch，每个patch的位置编码都是有明确意义的，如果patch数量变多，位置信息就会发生变化），一种做法是使用插值算法，扩大位置编码表。但是如果序列长度变化过大，插值操作会损失模型性能，这是ViT在微调时的一种局限性